In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.master("spark://cm1:7077").appName("testekafka").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/03 21:33:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/03 21:33:15 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/05/03 21:33:15 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/05/03 21:33:15 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [7]:
lines = spark.readStream.format("kafka") \
 .option("kafka.bootstrap.servers", "cm3:9092") \
 .option("subscribe", "student-a180052845-saida") \
 .option('includeTimestamp', 'true') \
 .load()

23/05/03 21:33:34 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources


In [43]:
from pyspark.sql.functions import explode, split, col, window

In [9]:
words = lines.select(explode(split(col("value"), "\s+")).alias("word"),lines.timestamp)

In [10]:
words.writeStream.format('console').option('truncate', 'false').start()

23/05/03 21:33:40 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-9e9a0204-0a0e-458c-81f2-647c8c116b2a. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/05/03 21:33:40 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


23/05/03 21:33:49 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources


-------------------------------------------
Batch: 0
-------------------------------------------
-------------------------------------------
Batch: 0
-------------------------------------------
+----+---------+
|word|timestamp|
+----+---------+
+----+---------+

+----+---------+
|word|timestamp|
+----+---------+
+----+---------+



-------------------------------------------
Batch: 1
-------------------------------------------
-------------------------------------------
Batch: 1
-------------------------------------------
+----+-----------------------+
|word|timestamp              |
+----+-----------------------+
|Gabi|2023-05-03 21:34:12.677|
+----+-----------------------+

+----+-----------------------+
|word|timestamp              |
+----+-----------------------+
|Gabi|2023-05-03 21:34:12.677|
+----+-----------------------+



In [18]:
df = spark.readStream \
        .format("socket") \
        .option("host","cm1") \
        .option("port","9044") \
        .option('includeTimestamp', 'true') \
        .load()

23/05/03 21:46:30 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [54]:
words = df.select(explode(split(col("value"), "\s+")).alias("word"),df.timestamp)
x = words.withWatermark("timestamp", "10 minutes") \
    .groupBy(
        window(words.timestamp, "10 minutes"),
        words.word) \
    .count()
x.writeStream.format('console').option('truncate', 'false').start()

23/05/03 22:39:53 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-8951e42f-ce5b-418a-bc64-be9de0c75bc5. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/05/03 22:39:53 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


AnalysisException: Append output mode not supported when there are streaming aggregations on streaming DataFrames/DataSets without watermark;
Aggregate [window#397, word#389], [window#397 AS window#392, word#389, count(1) AS count#396L]
+- Project [named_struct(start, precisetimestampconversion(((precisetimestampconversion(timestamp#138, TimestampType, LongType) - (((precisetimestampconversion(timestamp#138, TimestampType, LongType) - 0) + 600000000) % 600000000)) - 0), LongType, TimestampType), end, precisetimestampconversion((((precisetimestampconversion(timestamp#138, TimestampType, LongType) - (((precisetimestampconversion(timestamp#138, TimestampType, LongType) - 0) + 600000000) % 600000000)) - 0) + 600000000), LongType, TimestampType)) AS window#397, word#389, timestamp#138-T600000ms]
   +- Filter isnotnull(timestamp#138)
      +- EventTimeWatermark timestamp#138: timestamp, 10 minutes
         +- Project [word#389, timestamp#138]
            +- Generate explode(split(value#137, 

In [29]:
sc = spark.sparkContext
rdd1 = sc.parallelize(df)
countpalavras = rdd1.map(lambda x: (x,1))
count2 = countpalavras.reduceByKey(lambda x, y: x+y)

Traceback (most recent call last):
  File "/home/prof/hadoop/python/pyspark/serializers.py", line 458, in dumps
    return cloudpickle.dumps(obj, pickle_protocol)
  File "/home/prof/hadoop/python/pyspark/cloudpickle/cloudpickle_fast.py", line 73, in dumps
    cp.dump(obj)
  File "/home/prof/hadoop/python/pyspark/cloudpickle/cloudpickle_fast.py", line 602, in dump
    return Pickler.dump(self, obj)
TypeError: cannot pickle '_thread.RLock' object


PicklingError: Could not serialize object: TypeError: cannot pickle '_thread.RLock' object

In [56]:
words.writeStream.format('console').option('truncate', 'false').start()

23/05/03 22:43:08 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-0ea2fdfe-048d-4380-bff6-9aec9a1ee20c. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/05/03 22:43:08 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+---------+
|word|timestamp|
+----+---------+
+----+---------+

-------------------------------------------
Batch: 1
-------------------------------------------
+----+-----------------------+
|word|timestamp              |
+----+-----------------------+
|oii |2023-05-03 22:43:08.409|
+----+-----------------------+



-------------------------------------------
Batch: 2
-------------------------------------------
+----+-----------------------+
|word|timestamp              |
+----+-----------------------+
|td  |2023-05-03 22:43:18.902|
|bem?|2023-05-03 22:43:18.902|
+----+-----------------------+



In [62]:
print(words.word[1])

Column<'word[1]'>
